## OOI Seafloor Camera Open Datasets Example

This Jupyter notebook shows how to load video data from the [OOI Seafloor Camera](https://oceanobservatories.org/instrument-class/camhd/) system deployed at Axial Volcano on the Juan de Fuca Ridge. It demonstrates the basic usage of the pycamhd library, which can be used to extract frames from the ProRes encoded Quicktime files. These data are hosted on Microsoft's [Azure Open Datasets](https://azure.microsoft.com/en-us/services/open-datasets/catalog/ooi-camhd/).

### Imports and constants

In [ ]:
# Standard packages
import numpy as np
import pandas as pd
import fsspec
import time
import datetime
import random
import matplotlib.pyplot as plt
from ipywidgets import interact
from ipywidgets import IntSlider

# Non-standard, but still pip- or conda-installable
import pycamhd as camhd

# .json file containing video metadata
dbcamhd_url = 'https://ooiopendata.blob.core.windows.net/camhd/dbcamhd.json'

### Load the JSON file that lists all available video files
This file has useful information about each file in Azure Open Datasets, such as the Unix timestamp (seconds) of the first frame in each video, and the total number of frames in each video.

In [ ]:
with fsspec.open(dbcamhd_url) as f:
    dbcamhd = pd.read_json(f, orient="records", lines=True)
dbcamhd.tail()

### Choose a file to work with

In [ ]:
# Find files from September 20, 2017...
start_time = datetime.datetime(2017,9,20,0,0,0)
end_time = datetime.datetime(2017,9,21,0,0,0)

start_unixtime = time.mktime(start_time.timetuple())
end_unixtime = time.mktime(end_time.timetuple())

matching_rows = dbcamhd[dbcamhd['timestamp'].between(start_unixtime,end_unixtime)]
matching_rows

In [ ]:
# ...and choose the first file from that day.
mov = matching_rows.iloc[0]
mov

### Render one frame from the video

In [ ]:
def show_image(frame_number):
    plt.rc('figure', figsize=(12, 6))
    plt.rcParams.update({'font.size': 8})
    frame = camhd.get_frame(mov.url, frame_number)
    fig, ax = plt.subplots();
    im1 = ax.imshow(frame);
    plt.yticks(np.arange(0,1081,270))
    plt.xticks(np.arange(0,1921,480))
    plt.title('Deployment: %s    File: %s    Frame: %s' % (mov.deployment, mov['name'], frame_number));

# Choose a random frame 
initial_frame = random.randrange(0,mov.frame_count)
print('Showing frame {}'.format(initial_frame))
show_image(initial_frame)

### Interactively explore frames from the video
We include a slider which allows any frame from this video to be shown.  If you're viewing an HTML version of this notebook, this cell will look sad, but we swear, the widget you don't see allows you to interactively view frames.

In [ ]:
frame_slider = IntSlider(min=0, max=mov.frame_count-1, step=1, value=initial_frame, continuous_update=False)
interact(show_image, frame_number=frame_slider)